<a href="https://colab.research.google.com/github/anjali-mi/project/blob/main/AnnModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Necessary Libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l2
from datetime import datetime
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')


# Spliting the dataset into training, validation, and test sets (e.g., 70%-15%-15%).

In [ ]:
# # Donot Run this shell, On local It was performed, Not Need here
# import os
# import shutil
# from sklearn.model_selection import train_test_split

# def create_dir(path):
#     if not os.path.exists(path):
#         os.makedirs(path)

# # Directories
# input_folder = 'Datasets/'
# output_folder = 'datasets_split/'

# classes = [d for d in os.listdir(input_folder) if os.path.isdir(os.path.join(input_folder, d))]

# train_ratio = 0.7
# val_ratio = 0.15
# test_ratio = 0.15

# for class_name in classes:
#     class_path = os.path.join(input_folder, class_name)
#     images = os.listdir(class_path)

#     train_images, temp_images = train_test_split(images, test_size=(val_ratio + test_ratio), random_state=42)
#     val_images, test_images = train_test_split(temp_images, test_size=test_ratio/(val_ratio + test_ratio), random_state=42)

#     train_dir = os.path.join(output_folder, 'train', class_name)
#     val_dir = os.path.join(output_folder, 'val', class_name)
#     test_dir = os.path.join(output_folder, 'test', class_name)

#     create_dir(train_dir)
#     create_dir(val_dir)
#     create_dir(test_dir)

#     for image in train_images:
#         shutil.copy(os.path.join(class_path, image), os.path.join(train_dir, image))

#     for image in val_images:
#         shutil.copy(os.path.join(class_path, image), os.path.join(val_dir, image))

#     for image in test_images:
#         shutil.copy(os.path.join(class_path, image), os.path.join(test_dir, image))

# print("Dataset split completed successfully!")


Purpose

This script is used for preprocessing a dataset by splitting it into three subsets:


Training Set: Used to train the machine learning model.

Validation Set: Used to tune the model and validate it during training to prevent overfitting.

Test Set: Used to test the final model's performance.


Key Steps


Check and Create Directories: Ensures the necessary folder structure for saving the subsets.

Define Split Ratios: Specifies how the dataset should be divided (e.g., 70% for training, 15% for validation, 15% for testing).

Split the Data: Randomly divides the dataset into the three subsets based on the defined ratios.

Copy Files: Copies the respective files into their designated directories.




Outcome

By running this script, you get a well-organized dataset divided into training, validation, and test sets, which is crucial for:

Ensuring the model is trained on one part of the data.

Validating its performance during training on a separate part.

Evaluating its final performance on unseen data.

This structured approach is key to building effective and reliable machine learning models!


# Data augmentation

In [ ]:
# Parameters
input_shape = (128, 128, 3) # 128x128 pixels with 3 color channels (RGB)
num_classes = 5
target_size = (128, 128) # Target size to which all images will be resized
batch_size = 64 # Number of images to be processed in each batch during training, validation, and testing,

# Directories
train_dir = '/content/drive/MyDrive/AI ML GROUP PROJECT/Project2/datasets_split/train'
val_dir = '/content/drive/MyDrive/AI ML GROUP PROJECT/Project2/datasets_split/val'
test_dir = '/content/drive/MyDrive/AI ML GROUP PROJECT/Project2/datasets_split/test'

# Data Augmentation and Normalization for Training Data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

# Data Normalization for Validation and Test Data (only rescaling)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 965 images belonging to 5 classes.
Found 206 images belonging to 5 classes.
Found 209 images belonging to 5 classes.


**purpose**

This code snippet is focused on data preparation and augmentation for training, validating, and testing a machine learning model using images. The ultimate goal is to enhance the model's generalization ability by creating a robust and diverse training dataset.


**keys Components**

Parameters:

input_shape: Specifies the shape of the input images (128x128 pixels with 3 color channels).

num_classes: The number of different classes in the dataset.

target_size: Ensures all images are resized to 128x128 pixels.

batch_size: Defines the number of images processed in each batch during training, validation, and testing.


Directories:


Directories for training, validation, and test datasets are specified. These directories contain the images that will be used in each phase.


Data Augmentation and Normalization:


train_datagen: This object applies various augmentations (e.g., rescaling, rotation, shifting, shearing, zooming, flipping, brightness adjustment) to the training images to artificially expand the training dataset and make the model more robust.

val_test_datagen: This object only applies rescaling to the validation and test images for consistency.


Data Generators:


train_generator: Loads training images from the specified directory and applies the defined augmentations.

val_generator: Loads validation images and applies rescaling.

test_generator: Loads test images and applies rescaling.


Outcome

By running this code, you achieve several important outcomes:

Standardized Input: All images are resized to a common size (128x128 pixels) and normalized, making it easier for the model to process.

Enhanced Training Data: Data augmentation techniques generate more varied images from the existing dataset, helping the model to generalize better and reduce overfitting.

Efficient Data Loading: Data generators streamline the process of loading and preprocessing images in batches, optimizing memory usage and training speed.


This setup prepares your dataset in a way that maximizes the effectiveness of your machine learning model during training and evaluation phases.





# Architecture: Designing ANN Model

In [ ]:
def build_ann_model(input_shape, num_classes):
    model = Sequential([
        Flatten(input_shape=input_shape),
        Dense(256, activation='relu'),

        Dense(128, activation='relu'),

        Dense(64, activation='relu'),

        Dense(num_classes, activation='softmax')
    ])
    return model


**Purpose**

This function builds an Artificial Neural Network (ANN) model using Keras with the following layers:

Flatten: Converts the input images to a 1D array to feed into the dense layers.

Dense Layers: Adds three fully connected layers with 256, 128, and 64 neurons respectively, using ReLU activation to introduce non-linearity.

Output Layer: Adds a final dense layer with num_classes neurons and softmax activation to classify the input into one of the specified classes.

**Outcome**

After using this function, you get a compiled ANN model ready for training, capable of learning patterns from the input data and making predictions based on the number of classes specified.

# Training Artificial Neural Networks (ANNs) model

In [ ]:
# Function Definition

def train_model(optimizer, learning_rate, batch_size):

    # Optimizer Selection

    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
    else:
        raise ValueError("Unsupported optimizer")

  # Model Building and Compiling

    model = build_ann_model(input_shape, num_classes)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

      # Logging and Callbacks

    log_dir = f"/content/drive/MyDrive/AI ML GROUP PROJECT/Project2/logs/ANN_{optimizer}_{str(learning_rate).replace('.', '')}_{batch_size}_{datetime.now().strftime('%Y%m%d-%H%M%S')}"
    tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    checkpoint_callback = ModelCheckpoint(
        filepath='/content/drive/MyDrive/AI ML GROUP PROJECT/Project2/checkpoints/model-{epoch:02d}-{val_accuracy:.2f}.keras',
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    )
      # Model Training
        epochs=15,
        validation_data=val_generator,
        callbacks=[tensorboard_callback, early_stopping, checkpoint_callback],
        batch_size=batch_size
    )
      # Model Evaluation

    test_loss, test_acc = model.evaluate(test_generator)
    print(f'Test Accuracy for optimizer {optimizer}, learning rate {learning_rate}, batch size {batch_size}: {test_acc:.2f}')


**Purpose**


This function trains a machine learning model with flexibility in selecting optimizers, learning rates, and batch sizes. It incorporates various techniques to enhance training efficiency and model performance.


Key Steps
Optimizer Selection:


Based on the input parameter (optimizer), the function selects and configures the desired optimizer (Adam, SGD, RMSprop, Adagrad) with a specified learning rate.


Model Building and Compiling:


build_ann_model: Calls the function to create the ANN model.


model.compile: Compiles the model with the chosen optimizer, using categorical cross-entropy as the loss function and accuracy as a performance metric.

Logging and Callbacks:


TensorBoard: Sets up TensorBoard for visualizing training progress.

EarlyStopping: Stops training early if the validation loss does not improve for a specified number of epochs (patience), restoring the best weights.

ModelCheckpoint: Saves the model at each epoch if the validation accuracy improves, ensuring the best model is retained.

Model Training:

Trains the model for a specified number of epochs (epochs=15) using the training data (train_generator), with validation data (val_generator), and specified callbacks (tensorboard_callback, early_stopping, checkpoint_callback).

Model Evaluation:

Evaluates the model's performance on the test data (test_generator) and prints the test accuracy for the given optimizer, learning rate, and batch size.


Outcome

By using this function, you get a well-trained and evaluated model with optimized performance. The use of different optimizers and learning rates allows for experimentation and finding the best settings. The inclusion of callbacks ensures efficient training and model management.

This comprehensive approach is key to building and fine-tuning effective machine learning models!



# Experimentation

In [ ]:
# Experiment configurations
optimizers = ['adam', 'sgd', 'rmsprop', 'adagrad']
learning_rates = [0.001, 0.0001]
batch_sizes = [64, 128]

# Run experiments
for optimizer in optimizers:
    for learning_rate in learning_rates:
        for batch_size in batch_sizes:
            train_model(optimizer, learning_rate, batch_size)


Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 513ms/step - accuracy: 0.2424 - loss: 12.0638
Epoch 1: val_accuracy improved from -inf to 0.21845, saving model to /content/drive/MyDrive/AI ML GROUP PROJECT/Project2/checkpoints/model-01-0.22.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - accuracy: 0.2435 - loss: 12.1567 - val_accuracy: 0.2184 - val_loss: 12.6136
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 521ms/step - accuracy: 0.2820 - loss: 8.8245
Epoch 2: val_accuracy improved from 0.21845 to 0.36893, saving model to /content/drive/MyDrive/AI ML GROUP PROJECT/Project2/checkpoints/model-02-0.37.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.2846 - loss: 8.7195 - val_accuracy: 0.3689 - val_loss: 3.8033
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4235 - loss: 2.4524
Epoch 3: val_accuracy improved from 0.36893 to 0.53398, saving model to /content/drive/MyDrive/AI ML GROUP PROJECT/Project2/checkpoints/model-03-0.53.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accura

Purpose

This code snippet is designed to run multiple experiments by systematically varying key hyperparameters, such as optimizers, learning rates, and batch sizes. The goal is to find the best combination of these hyperparameters that results in the highest model performance.

Key Components

Optimizers: Different optimization algorithms like 'adam', 'sgd', 'rmsprop', and 'adagrad'.

Learning Rates: The step size at each iteration while moving toward a minimum of the loss function. Here, the values are 0.001 and 0.0001.

Batch Sizes: Number of training examples utilized in one iteration. The values are 64 and 128.

Steps

Iterate through Configurations:

Nested loops go through each combination of optimizer, learning rate, and batch size.

Train Model:


For each combination, the train_model function is called with the specific hyperparameters.

The function trains the model, logs training details, and evaluates the performance on the test set.


Outcome


By running these experiments, you can:

Compare Different Configurations: Identify which combination of hyperparameters yields the best model performance.

Optimize Model: Fine-tune your model to achieve better accuracy and generalization.



This systematic approach allows you to empirically determine the best setup for your specific machine learning task. This kind of hyperparameter tuning is essential for developing high-performance models.





# Failed Tensorboard launch

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/AI ML GROUP PROJECT/Project2/datasets_split/logs


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2024-12-07 16:59:44.263390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 16:59:44.288371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 16:59:44.295457: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 16:59:45.599974: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC] [--host ADDR]
                   [--bind_all] [--port PORT] [--reuse_port BOOL] [--load_f

Purpose

This snippet is used to load TensorBoard extension within a Jupyter notebook environment and launch TensorBoard to visualize training logs.

Steps
load_ext tensorboard:


Loads the TensorBoard extension into the notebook environment, allowing the use of TensorBoard magic commands.

%tensorboard --logdir:


Starts TensorBoard and points it to the specified log directory (/content/drive/MyDrive/AI ML GROUP PROJECT/Project2/datasets_split/logs).

This directory contains the logs generated during model training, which can be visualized to monitor training progress.


Outcome

By running these commands, you can:

Visualize Training Metrics: View metrics like loss, accuracy, and others over the training epochs.

Gain Insights: Understand how well the model is performing and identify any issues like overfitting or underfitting.


TensorBoard provides a powerful and interactive way to monitor and analyze the training process, helping to fine-tune and optimize your model effectively.



